In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm

In [ ]:
# Define the file path and names
base_path = ""
locations = ['Shinagawa','Kita Senju','Oshiage','Tamachi', 'Hamamatsucho','Ayase','Meguro','Ichigaya','Yotsuya','Kotake Mukaihara']

# Prepare the dataframes to store combined results
combined_decision_variables = pd.DataFrame()
combined_average_areas = pd.DataFrame()
combined_final_area = pd.DataFrame()
combined_price_layers = pd.DataFrame()

# Function to read data from excel files and add file info
def read_excel_sheets(file_path, location, file_type):
    decision_variables = pd.read_excel(file_path, sheet_name='Decision_Variables')
    decision_variables['Location'] = location
    decision_variables['File_Type'] = file_type

    average_areas = pd.read_excel(file_path, sheet_name='Average_Areas')
    average_areas['Location'] = location
    average_areas['File_Type'] = file_type

    final_area = pd.read_excel(file_path, sheet_name='Final_Area')
    final_area['Location'] = location
    final_area['File_Type'] = file_type

    return decision_variables, average_areas, final_area

# Iterate over each location and read the corresponding files
for location in locations:
    # Read Simple_number_change_optimization_results files
    simple_file = os.path.join(base_path, f"Simple_number_change_optimization_results_{location}.xlsx")
    if os.path.exists(simple_file):
        decision_variables, average_areas, final_area = read_excel_sheets(simple_file, location, "Simple_number_change_optimization_results")
        combined_decision_variables = pd.concat([combined_decision_variables, decision_variables])
        combined_average_areas = pd.concat([combined_average_areas, average_areas])
        combined_final_area = pd.concat([combined_final_area, final_area])
    
    # Read optimization_results files
    optimization_file = os.path.join(base_path, f"optimization_results_{location}.xlsx")
    if os.path.exists(optimization_file):
        decision_variables, average_areas, final_area = read_excel_sheets(optimization_file, location, "optimization_results")
        combined_decision_variables = pd.concat([combined_decision_variables, decision_variables])
        combined_average_areas = pd.concat([combined_average_areas, average_areas])
        combined_final_area = pd.concat([combined_final_area, final_area])
    
    # Read Price_optimization_results files
    price_file = os.path.join(base_path, f"Price_optimization_results_{location}.xlsx")
    if os.path.exists(price_file):
        decision_variables, average_areas, final_area = read_excel_sheets(price_file, location, "Price_optimization_results")
        price_layers = pd.read_excel(price_file, sheet_name='Price_Layers')
        price_layers['Location'] = location
        price_layers['File_Type'] = "Price_optimization_results"
        
        combined_decision_variables = pd.concat([combined_decision_variables, decision_variables])
        combined_average_areas = pd.concat([combined_average_areas, average_areas])
        combined_final_area = pd.concat([combined_final_area, final_area])
        combined_price_layers = pd.concat([combined_price_layers, price_layers])

# Write the combined results into a new Excel file
output_path = os.path.join(base_path, "combined_optimization_results_with_file_info.xlsx")
with pd.ExcelWriter(output_path) as writer:
    combined_decision_variables.to_excel(writer, sheet_name='Decision_Variables')
    combined_average_areas.to_excel(writer, sheet_name='Average_Areas')
    combined_final_area.to_excel(writer, sheet_name='Final_Area')
    if not combined_price_layers.empty:
        combined_price_layers.to_excel(writer, sheet_name='Price_Layers')

output_path


In [ ]:
# Visulization
file_path = 'Figure_combined_optimization_results_with_file_info.xlsx'
data = pd.read_excel(file_path, sheet_name='Average_Areas_re')
data = data.iloc[0:10, 0:7]
data.set_index(data.columns[0], inplace=True)
display(data)

# Convert DataFrame to dictionary format
data_dict = {row[0]: list(row[1:]) for row in data.itertuples(index=False)}

# Function to draw a pentagon
def draw_pentagon(values, ax, label):
    categories = ['Nutrition_Score','Price_Score','Carbon_Score','Diversity','Crowdedness']
    
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist() + [0]
    values = values[:len(categories)]  # Ensure only the required number of values
    values += values[:1]  # Complete the loop for the pentagon
    
    ax.plot(angles, values, 'o-', linewidth=2, color='gray')
    ax.fill(angles, values, alpha=0.25, color='gray')
    ax.set_ylim(0, 1.1)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.set_title(label, size=10, fontname='Arial')

    # Add colored vertices
    colors = ['#E97132', '#C00000', '#FFC000', '#28A7CC', '#353C83']
    for angle, value, color in zip(angles[:-1], values[:-1], colors):
        ax.plot([angle], [value], marker='o', markersize=8, color=color)

# Create a 5x4 subplot
fig, axs = plt.subplots(5, 4, figsize=(15, 15), subplot_kw=dict(polar=True))

# Flatten the array of axes
axs = axs.flatten()

# Plot each pentagon
for ax, (label, values) in zip(axs, data_dict.items()):
    draw_pentagon(values, ax, label)

    
# Hide any remaining subplots
for i in range(len(data_dict), len(axs)):
    axs[i].set_visible(False)

plt.tight_layout()

# Save the figure as PNG
plt.savefig('Radar_colored.svg', format='svg')

plt.show()


In [ ]:
# Set the font and color globally
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'

# Load the data
file_path = 'Figure_combined_optimization_results_with_file_info.xlsx'
data = pd.read_excel(file_path, sheet_name='restaurant_distribution_re')

# Define the color palette
colors = [
    "#23A7CC",  # 深青色 for 0
    "#A0CCDB",  # 青色 for 1-5
    "#D3EBEA",  # 浅青色 for 6-10
    "#F5E7CF",  # 浅黄色 for 11-20
    "#F4B9AC",  # 淡黄色 for 21-60
    "#D0636C"   # 淡橙色 for >60
]

# Create a custom colormap
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)

# Define boundaries and norm
bounds = [0, 5, 20, 40, 65, 90, 150]
norm = BoundaryNorm(bounds, cmap.N)

# Function to plot heatmap for a specific location
def plot_location_heatmap(location, data, ax, show_y_label=False, show_x_label=False):
    # Filter data for the specific location
    location_data = data[data['Location'] == location]
    
    # Drop 'Location' column as it's redundant now
    location_data = location_data.drop(columns=['Location'])
    # Remove columns (restaurant types) with all zeros
    location_data = location_data.loc[(location_data != 0).any(axis=1)]
        # Rename the models
    model_rename = {
        'Model 2': 'NT Scenario',
        'Model 1': 'TDO Scenario',
        'Model 3': 'PRC Scenario'
    }
    location_data['File_Type'] = location_data['File_Type'].replace(model_rename)
    
    # Set 'File_Type' as index
    location_data = location_data.set_index('File_Type')
    
    # Set 'File_Type' as index
#     location_data = location_data.set_index('File_Type')
    
    # Transpose the data for heatmap
    location_data = location_data.T
    
    
    # Reorder the rows according to the specified sequence
    order = ['BAU', 'NT Scenario', 'TDO Scenario', 'PRC Scenario']
    

    sns.heatmap(
        location_data.T,  # Transpose to rotate
        cmap=cmap, 
        norm=norm,
        linewidths=.5, 
        annot=True, 
        fmt='d',
        cbar_kws={'label': 'Number of Restaurants'},
        ax=ax
    )
    
    # Set title
    ax.set_title(location)
    
    # Show or hide x and y labels
    if show_y_label:
        ax.set_ylabel('')
        ax.tick_params(left=True, bottom=False)
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    else:
        ax.set_ylabel('')
        ax.tick_params(left=False, bottom=False)
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    if show_x_label:
        ax.set_xlabel('')
        ax.tick_params(left=False, bottom=True)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='center')
    else:
        ax.set_xlabel('')
        ax.tick_params(left=False, bottom=False)

# Get the unique locations
locations = data['Location'].unique()

# Create a figure with subplots
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(18, 15))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot heatmaps for each location
for i, location in enumerate(locations):
    show_y_label = i % 2 == 0  # Show y labels for the left column
    show_x_label = i >= (len(locations) - 2)  # Show x labels for the bottom row
    plot_location_heatmap(location, data, axes[i], show_y_label, show_x_label)

# Adjust layout to make space between plots
plt.tight_layout()

# Save the plot as SVG
plt.savefig('combined_heatmaps.svg', format='svg')

# Show the plot
plt.show()
